In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import imageio
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imageio
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imageio
import os

In [9]:
df = pd.read_csv('tracking_week_1.csv')
df.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN


In [3]:
df.groupby(["gameId"])["playId"].count()

gameId
2022090800     79212
2022091100     93587
2022091101     92552
2022091102    107019
2022091103     91080
2022091104     99452
2022091105    108330
2022091106     75049
2022091107     83030
2022091108     90137
2022091109     78775
2022091110     93173
2022091111     77648
2022091112     69115
2022091113     85376
2022091200     83904
Name: playId, dtype: int64

In [4]:
df.groupby(["gameId", "playId"])["jerseyNumber"].count()

gameId      playId
2022090800  56         484
            80         660
            101       1078
            122        726
            146       1232
                      ... 
2022091200  3628       704
            3685      1276
            3723      1408
            3747       506
            3826      1166
Name: jerseyNumber, Length: 1473, dtype: int64

In [5]:
df.groupby(["gameId", "playId","club"])["jerseyNumber"].count()

gameId      playId  club    
2022090800  56      BUF         242
                    LA          242
                    football      0
            80      BUF         330
                    LA          330
                               ... 
2022091200  3747    SEA         253
                    football      0
            3826    DEN         583
                    SEA         583
                    football      0
Name: jerseyNumber, Length: 4419, dtype: int64

# **Create Football Field**

In [14]:
# Function to create the football field
def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax

# **Live Scatter**

In [15]:
# Filter for a single play
play_df = df[(df['gameId'] == 2022090800) & (df['playId'] == 122)]

# Find the unique frames in the play
unique_frames = play_df['frameId'].unique()

# Define colors for the teams
team_colors = {'BUF': 'blue', 'LA': 'yellow'}

# Plot each frame and collect image filenames
image_filenames = []

for frame in unique_frames:
    frame_df = play_df[play_df['frameId'] == frame]

    # Create the football field
    fig, ax = create_football_field()

    # Plot player positions with team colors
    for team, color in team_colors.items():
        team_df = frame_df[frame_df['club'] == team]
        ax.scatter(team_df['x'], team_df['y'], c=color, label=team, zorder=2)

    # Annotate each point with the player's jersey number, if it's not NaN
    for _, row in frame_df.iterrows():
        if pd.notna(row['jerseyNumber']):
            ax.annotate(int(row['jerseyNumber']), (row['x'], row['y']), color='white', zorder=3)

    # Optional: Add a legend if you want to label the colors
    ax.legend(loc='upper right', frameon=False)

    # Save the current plot frame
    filename = f'frame_{frame}.png'
    plt.savefig(filename, bbox_inches='tight')
    image_filenames.append(filename)
    plt.close(fig)

# Create an animated GIF
images = [imageio.imread(filename) for filename in image_filenames]
imageio.mimsave('player_movement_by_team_on_field.gif', images, fps=10)

# Clean up by removing the saved image files
for filename in image_filenames:
    os.remove(filename)


C:\Users\decla\AppData\Local\Temp\ipykernel_22448\2428840515.py:39: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(filename) for filename in image_filenames]


# **Tracked Scatter**

In [16]:
# Filter for a single play
play_df = df[(df['gameId'] == 2022090800) & (df['playId'] == 122)]

# Find the unique frames in the play
unique_frames = sorted(play_df['frameId'].unique())

# Define colors for the teams
team_colors = {'BUF': 'blue', 'LA': 'yellow'}

# Prepare player trails data structure, ignoring NaN nflIds
player_trails = {nflId: {'x': [], 'y': [], 'team': None} for nflId in play_df['nflId'].dropna().unique()}

# Plot each frame and collect image filenames
image_filenames = []

for frame in unique_frames:
    frame_df = play_df[play_df['frameId'] == frame]
    fig, ax = create_football_field()

    # Update trails and plot for each player, ignoring rows with NaN nflId
    for _, row in frame_df.iterrows():
        nflId = row['nflId']
        if pd.notna(nflId):  # Check if nflId is not NaN
            player_trails[nflId]['x'].append(row['x'])
            player_trails[nflId]['y'].append(row['y'])
            if player_trails[nflId]['team'] is None:
                player_trails[nflId]['team'] = row['club']

    # Draw player trails up to the current frame
    for nflId, data in player_trails.items():
        team = data['team']
        color = team_colors.get(team, 'white')  # Fallback color if team not in dictionary
        ax.plot(data['x'], data['y'], c=color, zorder=2)

    # Save the current plot frame
    filename = f'frame_{frame}.png'
    plt.savefig(filename, bbox_inches='tight')
    image_filenames.append(filename)
    plt.close(fig)

# Create an animated GIF
images = [imageio.imread(filename) for filename in image_filenames]
imageio.mimsave('player_movement_trails.gif', images, fps=10)

# Clean up by removing the saved image files
for filename in image_filenames:
    os.remove(filename)


C:\Users\decla\AppData\Local\Temp\ipykernel_22448\4139484800.py:42: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(filename) for filename in image_filenames]
